In [17]:
import pandas as pd

In [26]:
data = pd.read_csv('Data for UC#29 and 27.csv')

In [27]:
data.drop(data.tail(3).index,inplace=True)

In [28]:
data

,Test Case ID,Test Run Id,Failure Step count (since inception),Failed Steps,Exception Name,Failed Step update date,Execution Time for test case,Predecessor TestCase ID,Successor TestCase ID
0,457.0,122.0,1.0,7.0,Fails to navigate step2,10-Jul-19,16:22.3,0.0,405.0
1,405.0,122.0,1.0,1.0,"Verification Failed. Expected text: ""Manual Ha...",10-Jul-19,11:49.1,457.0,371.0
2,371.0,122.0,1.0,3.0,"Verification Failed. Expected text: ""Step 6 of...",10-Jul-19,54:29.2,405.0,370.0
3,370.0,122.0,1.0,6.0,"Verification Failed. Expected text: ""Step 6 of...",10-Jul-19,58:18.5,371.0,369.0
4,369.0,122.0,1.0,10.0,"Verification Failed. Expected text: ""Step 6 of...",10-Jul-19,02:25.1,370.0,431.0
...,...,...,...,...,...,...,...,...,...
128,341.0,122.0,1.0,4.0,Unable to find object with specified locator (...,11-Jul-19,21:05.6,320.0,348.0
129,348.0,122.0,1.0,8.0,Unable to find object with specified locator (...,11-Jul-19,32:49.8,341.0,351.0
130,351.0,122.0,1.0,0.0,Unable to find object with specified locator (...,11-Jul-19,34:37.6,348.0,350.0
131,350.0,122.0,1.0,0.0,Unable to find object with specified locator (...,11-Jul-19,40:56.3,351.0,336.0


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133 entries, 0 to 132
Data columns (total 9 columns):
Test Case ID                            133 non-null float64
Test Run Id                             133 non-null float64
Failure Step count (since inception)    133 non-null float64
Failed Steps                            133 non-null float64
Exception Name                          133 non-null object
Failed Step update date                 133 non-null object
Execution Time for test case            133 non-null object
Predecessor TestCase ID                 133 non-null float64
Successor TestCase ID                   133 non-null float64
dtypes: float64(6), object(3)
memory usage: 10.4+ KB


In [30]:
data.isna().sum()

Test Case ID                            0
Test Run Id                             0
Failure Step count (since inception)    0
Failed Steps                            0
Exception Name                          0
Failed Step update date                 0
Execution Time for test case            0
Predecessor TestCase ID                 0
Successor TestCase ID                   0
dtype: int64

In [32]:
data.duplicated().sum()

0

In [36]:
import datetime as dt

In [35]:
data['Failed Step update date'].dtypes

dtype('O')

In [38]:
data['month'] = data['Execution Time for test case'].dt.month
#data['day'] = data['Failed Step update date'].dt.day
#data['year'] = data['Failed Step update date'].dt.year

AttributeError: Can only use .dt accessor with datetimelike values

# Text Wrangling and Normalization

In [55]:
from bs4 import BeautifulSoup
import re
import tqdm
import unicodedata

In [56]:
#function defined to remove html tags from data

def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

In [57]:
#function defined to remove unicode data

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

In [42]:
# lower case and remove special characters\whitespaces

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    #doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()  
    norm_docs.append(doc)
  
  return norm_docs

In [43]:
#Data_Preprocessing

norm_exception = pre_process_corpus(data['Exception Name'])


100%|██████████| 133/133 [00:00<00:00, 1878.36it/s]


In [44]:
norm_exception

['fails to navigate step2',
 'verification failed expected text manual handling not found in element',
 'verification failed expected text step 6 of 6 save and assign claim not found in element',
 'verification failed expected text step 6 of 6 save and assign claim not found in element',
 'verification failed expected text step 6 of 6 save and assign claim not found in element',
 'verification failed expected text straight payment not found in element',
 'verification failed expected text step 6 of 6 save and assign claim not found in element',
 'verification failed expected text step 6 of 6 save and assign claim not found in element',
 'verification failed expected text step 4 of 6 add costs information not found in element',
 'unable to find object with specified locator nametripgenericexpensepopupclaimgenetricexpencedetaildvclaimtripotherdetaildvcompanyname',
 'verification failed expected text step 6 of 6 save and assign claim not found in element',
 'verification failed expected t

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

In [46]:
# build BOW features on train reviews
cv = CountVectorizer(binary=False, min_df=5, max_df=1.0, ngram_range=(1,2))
cv_train_features = cv.fit_transform(norm_exception)

In [47]:
print (cv.get_feature_names())

['add', 'add claim', 'add costs', 'and', 'and assign', 'and stored', 'assign', 'assign claim', 'been', 'been created', 'claim', 'claim information', 'claim not', 'costs', 'costs information', 'created', 'created and', 'element', 'expected', 'expected text', 'failed', 'failed expected', 'fails', 'fails to', 'find', 'find object', 'found', 'found in', 'has', 'has been', 'idfnolwizardnext', 'in', 'in element', 'info', 'information', 'information not', 'locator', 'locator idfnolwizardnext', 'locator namefnolwizardfullwizardstepsetfnolwizardcreditcardtcoscreenotherinsurancecompanytcodvinsurancecompany', 'locator nametripaccommodationaddresspopupclaimpolicytripaccommodationdetaildvclaimtripaccommodationdetaildvdescription', 'namefnolwizardfullwizardstepsetfnolwizardcreditcardtcoscreenotherinsurancecompanytcodvinsurancecompany', 'nametripaccommodationaddresspopupclaimpolicytripaccommodationdetaildvclaimtripaccommodationdetaildvdescription', 'navigate', 'navigate step2', 'not', 'not found', 'o

In [48]:
print(cv_train_features.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 1 1]]


In [49]:
print('BOW model:> Train features shape:', cv_train_features.shape)

BOW model:> Train features shape: (133, 76)


In [29]:
cv_train_features

<450x925 sparse matrix of type '<class 'numpy.int64'>'
	with 15707 stored elements in Compressed Sparse Row format>

In [51]:

from sklearn.cluster import KMeans

NUM_CLUSTERS = 6
km = KMeans(n_clusters=NUM_CLUSTERS, max_iter=10000, n_init=100, random_state=42).fit(cv_train_features)
km

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=10000,
       n_clusters=6, n_init=100, n_jobs=None, precompute_distances='auto',
       random_state=42, tol=0.0001, verbose=0)

In [52]:
from collections import Counter

Counter(km.labels_)

Counter({2: 16, 4: 17, 0: 52, 3: 12, 1: 30, 5: 6})

In [54]:
data['kmeans_cluster'] = km.labels_

In [33]:

cat_cluster = (data[['REVIEW', 'kmeans_cluster', 'Module']]
                  .sort_values(by=['kmeans_cluster'], 
                               ascending=False)
                  .groupby('kmeans_cluster').head(20))

              precision    recall  f1-score   support

    Negative       0.96      0.93      0.95       297
    Positive       0.56      0.74      0.64        38

    accuracy                           0.90       335
   macro avg       0.76      0.83      0.79       335
weighted avg       0.92      0.90      0.91       335



,negative,positive
negative,275,22
positive,10,28


In [34]:
#TestData Validation

TestData = pd.read_csv('TestReviews.csv')

In [35]:
TestData

,Review,Sentiment
0,Won't let me check in. I thought the most rece...,NaN
1,I type in the booking reference and the app cr...,NaN
2,"This new App sucks, I am a frequent flyer with...",NaN
3,Doesn't work... won't find my reservations... ...,NaN
4,Nov 2019 Unstable app! So many crashes. So har...,NaN
5,"Just go to the mobile website, this app sucks....",NaN
6,"Error - 1022 We can't search for flights, beca...",NaN
7,Even if someone hires an app developer and pay...,NaN
8,11/30/19 Update: This update caused a duplicat...,NaN
9,OK... I upgraded the rating to 3 stars. To the...,NaN


In [36]:
#Data Pre-processing

reviews = TestData['Review'].values

norm_reviews = pre_process_corpus(reviews)

cv_test_data = cv.transform(norm_reviews)

100%|██████████| 50/50 [00:00<00:00, 2794.04it/s]


In [37]:
lr_test_predictions = lr.predict(cv_test_data)

In [38]:
lr_test_predictions

array(['Negative', 'Negative', 'Positive', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Positive',
       'Negative', 'Negative', 'Negative', 'Positive', 'Negative',
       'Negative', 'Positive', 'Positive', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Positive',
       'Negative', 'Negative', 'Negative', 'Positive', 'Negative',
       'Negative', 'Negative', 'Positive', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Positive',
       'Positive', 'Negative', 'Negative', 'Positive', 'Positive',
       'Negative', 'Positive', 'Negative', 'Negative', 'Positive'],
      dtype=object)

In [39]:
test_pred = pd.DataFrame(lr_test_predictions,columns= ["Sentiment"])
pred_result = pd.concat([TestData[["Review"]],test_pred] ,axis = 1)
pred_result.to_csv('PredictedResult.csv',index = False)

In [40]:
pred_result['Sentiment'].value_counts()

Negative    36
Positive    14
Name: Sentiment, dtype: int64

In [41]:

# Random Forest model on BOW features
from sklearn.ensemble import RandomForestClassifier

# instantiate model
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)

# train model
rf.fit(cv_train_features, train_sentiments)

# predict on test data
rf_bow_predictions = rf.predict(cv_test_features)

In [42]:
rf_bow_predictions

array(['Positive', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Positive', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Positive',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Positive', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negati

In [43]:
labels = ['', '']
print(classification_report(test_sentiments, rf_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, rf_bow_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

    Negative       0.95      0.97      0.96       297
    Positive       0.71      0.63      0.67        38

    accuracy                           0.93       335
   macro avg       0.83      0.80      0.81       335
weighted avg       0.93      0.93      0.93       335



,negative,positive
negative,287,10
positive,14,24


In [44]:
#Predict on Test Data

rf_test_predictions = rf.predict(cv_test_data)

In [45]:
rf_pred = pd.DataFrame(rf_test_predictions,columns= ["Sentiment"])
rf_pred_result = pd.concat([TestData[["Review"]],test_pred] ,axis = 1)
rf_pred_result.to_csv('PredictedResult.csv',index = False)

In [46]:
rf_pred_result

,Review,Sentiment
0,Won't let me check in. I thought the most rece...,Negative
1,I type in the booking reference and the app cr...,Negative
2,"This new App sucks, I am a frequent flyer with...",Positive
3,Doesn't work... won't find my reservations... ...,Negative
4,Nov 2019 Unstable app! So many crashes. So har...,Negative
5,"Just go to the mobile website, this app sucks....",Negative
6,"Error - 1022 We can't search for flights, beca...",Negative
7,Even if someone hires an app developer and pay...,Negative
8,11/30/19 Update: This update caused a duplicat...,Negative
9,OK... I upgraded the rating to 3 stars. To the...,Positive


In [47]:
#TestData Validation - ReviewsData_Fly Delta

Delta_Test = pd.read_csv('ReviewsData_FlyDelta.csv')

print (Delta_Test)

#Data Pre-processing

del_reviews = Delta_Test['Review'].values

norm_del_reviews = pre_process_corpus(del_reviews)

cv_delta = cv.transform(norm_del_reviews)

#Prediction on Test Data

lr_delta_predictions = lr.predict(cv_delta)



                                                Review Unnamed: 1
0    Myself and 2 girls went to check in and realiz...        NaN
1    OMG. I did it so on October 23 this was my ver...        NaN
2    Once the flight attendant Kara and Jay found o...        NaN
3    Our last 2 experiences with Delta have really ...        NaN
4    Since 2009 I have flown to Philippines for per...        NaN
..                                                 ...        ...
201  Useless PoC. You can immediately tell it never...        NaN
202  Don't log in as guest, as it may forget your b...        NaN
203  This app just randomly loses your data, and cr...        NaN
204        Boarding passes only show up half the time.        NaN
205  i wish there was a button when youre on your w...        NaN

[206 rows x 2 columns]


100%|██████████| 206/206 [00:00<00:00, 3149.71it/s]


In [48]:
#print (lr_delta_predictions)

lr_pred = pd.DataFrame(lr_delta_predictions,columns= ["Sentiment"])

lr_pred

,Sentiment
0,Negative
1,Negative
2,Positive
3,Negative
4,Positive
...,...
201,Negative
202,Positive
203,Positive
204,Negative


In [49]:
df_pred_result = pd.concat([Delta_Test[["Review"]],lr_pred] ,axis = 1)
df_pred_result.to_csv('Lr_DeltaResult.csv',index = False)

df_pred_result

,Review,Sentiment
0,Myself and 2 girls went to check in and realiz...,Negative
1,OMG. I did it so on October 23 this was my ver...,Negative
2,Once the flight attendant Kara and Jay found o...,Positive
3,Our last 2 experiences with Delta have really ...,Negative
4,Since 2009 I have flown to Philippines for per...,Positive
...,...,...
201,Useless PoC. You can immediately tell it never...,Negative
202,"Don't log in as guest, as it may forget your b...",Positive
203,"This app just randomly loses your data, and cr...",Positive
204,Boarding passes only show up half the time.,Negative


In [50]:
#Predict on Test Data

rf_test_delta = rf.predict(cv_delta)

rf_pred_delta = pd.DataFrame(rf_test_delta,columns= ["Sentiment"])

rf_pred_delta = pd.concat([Delta_Test[["Review"]],lr_pred] ,axis = 1)
rf_pred_delta.to_csv('rf_deltaResult.csv',index = False)


In [51]:
rf_pred_delta

,Review,Sentiment
0,Myself and 2 girls went to check in and realiz...,Negative
1,OMG. I did it so on October 23 this was my ver...,Negative
2,Once the flight attendant Kara and Jay found o...,Positive
3,Our last 2 experiences with Delta have really ...,Negative
4,Since 2009 I have flown to Philippines for per...,Positive
...,...,...
201,Useless PoC. You can immediately tell it never...,Negative
202,"Don't log in as guest, as it may forget your b...",Positive
203,"This app just randomly loses your data, and cr...",Positive
204,Boarding passes only show up half the time.,Negative


In [52]:
#TestData Validation - Aircanada

AirC_Test = pd.read_csv('AirCanada2.csv', encoding = "ISO-8859-1")

print (AirC_Test)

#Data Pre-processing

can_reviews = AirC_Test['Review'].values

norm_can_reviews = pre_process_corpus(can_reviews)

cv_airc = cv.transform(norm_can_reviews)

#Prediction on Test Data

lr_airc_predictions = lr.predict(cv_airc)

                                                Review
0    Update : Now that the tech issues have been ad...
1    Changing my rating since the problems are fixe...
2    App worked pretty much as expected. Could have...
3    Highly recommend when you travel with Air Cana...
4    The app went from very useful to absolutely us...
..                                                 ...
195  the app will say it has no connection if it is...
196  While waiting as a restuarant for my order, I ...
197  I'm having issues recently. The error I get is...
198  Some things work well. But the chekin process ...
199  I am disappointed with this app! good thing i ...

[200 rows x 1 columns]


100%|██████████| 200/200 [00:00<00:00, 2476.37it/s]


In [53]:
lr_airc_predictions

array(['Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positi

In [54]:
lr_can_pred = pd.DataFrame(lr_airc_predictions,columns= ["Sentiment"])

lr_can_pred

,Sentiment
0,Positive
1,Positive
2,Positive
3,Positive
4,Positive
...,...
195,Negative
196,Negative
197,Negative
198,Negative


In [55]:
lr_pred_airc = pd.concat([AirC_Test[["Review"]],lr_can_pred] ,axis = 1)
lr_pred_airc.to_csv('lr_canResult.csv',index = False)

In [56]:
lr_pred_airc['Sentiment'].value_counts()

Positive    126
Negative     74
Name: Sentiment, dtype: int64

In [57]:
#TestData Validation - FlyDelta

Del_Test = pd.read_csv('FlyDelta2.csv', encoding = "ISO-8859-1")

Del_Test

#print (Del_Test)

#Data Pre-processing

del1_reviews = Del_Test['Review'].values

norm_del1_reviews = pre_process_corpus(del1_reviews)

cv_delta1 = cv.transform(norm_del1_reviews)

#Prediction on Test Data

lr_del1_predictions = lr.predict(cv_delta1)

100%|██████████| 55/55 [00:00<00:00, 1501.08it/s]


In [58]:
lr_del1_pred = pd.DataFrame(lr_del1_predictions,columns= ["Sentiment"])


In [59]:
lr_pred_del = pd.concat([Del_Test[["Review"]],lr_del1_pred] ,axis = 1)

In [60]:
lr_pred_del.to_csv('lr_delResult.csv',index = False)
lr_pred_del['Sentiment'].value_counts()

Negative    37
Positive    18
Name: Sentiment, dtype: int64

In [61]:
lr_pred_del

,Review,Sentiment
0,Works well most of the time. Would be better w...,Negative
1,Really cool app. Used it for my trip to and fr...,Negative
2,The app was great but the update often shows n...,Positive
3,Often flight info just disappears forcing to u...,Negative
4,Not bad does well with standard flights but on...,Positive
5,The app continues to get better! When will you...,Negative
6,The app continues to get better! When will you...,Negative
7,Could use some improvements but probably the b...,Positive
8,"Latest update wouldn't open, uninstalled and r...",Positive
9,The app is okay and does its duty... **Navigat...,Positive


In [62]:
#Predict on Test Data

rf_test_del1 = rf.predict(cv_delta1)

rf_pred_del1 = pd.DataFrame(rf_test_del1,columns= ["Sentiment"])

rf_pred_delta1 = pd.concat([Del_Test[["Review"]],rf_pred_del1] ,axis = 1)
rf_pred_delta1.to_csv('rf_del1taResult.csv',index = False)
rf_pred_delta1['Sentiment'].value_counts()

Negative    50
Positive     5
Name: Sentiment, dtype: int64

In [63]:

from sklearn.feature_extraction.text import TfidfVectorizer

In [64]:
# build TFIDF features on train reviews
tv = TfidfVectorizer(use_idf=True, min_df=5, max_df=1.0, ngram_range=(1,2),
                     sublinear_tf=True)
tv_train_features = tv.fit_transform(norm_train_reviews)

In [65]:
tv_test_features = tv.transform(norm_test_reviews)

In [66]:
print('TFIDF model:> Train features shape:', tv_train_features.shape, ' Test features shape:', tv_test_features.shape)

TFIDF model:> Train features shape: (450, 925)  Test features shape: (335, 925)


In [67]:
# Logistic Regression model on TF-IDF features

# train model
lr.fit(tv_train_features, train_sentiments)

# predict on test data
lr_tfidf_predictions = lr.predict(tv_test_features)

labels = ['negative', 'positive']
print(classification_report(test_sentiments, lr_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, lr_tfidf_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

    Negative       0.93      0.99      0.96       297
    Positive       0.84      0.42      0.56        38

    accuracy                           0.93       335
   macro avg       0.89      0.71      0.76       335
weighted avg       0.92      0.93      0.91       335



,negative,positive
negative,294,3
positive,22,16


In [68]:
tv_test_data = tv.transform(norm_reviews)

In [69]:
lr_test_tfpred = lr.predict(tv_test_data)

In [70]:
lr_test_tfpred

array(['Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Positive', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative',
       'Negative', 'Negative', 'Negative', 'Negative', 'Negative'],
      dtype=object)

In [71]:
test_tfpred = pd.DataFrame(lr_test_tfpred,columns= ["Sentiment"])
tfpred_result = pd.concat([TestData[["Review"]],test_tfpred] ,axis = 1)
#pred_result.to_csv('PredictedResult.csv',index = False)

In [72]:
tfpred_result['Sentiment'].value_counts()

Negative    49
Positive     1
Name: Sentiment, dtype: int64

In [73]:
# Gradientboost Classifier
from sklearn.ensemble import GradientBoostingClassifier

In [74]:
gbc_fit = GradientBoostingClassifier(loss='deviance',learning_rate=0.05,n_estimators=5000,
                                     min_samples_split=2,min_samples_leaf=1,max_depth=1,random_state=42 )
gbc_fit.fit(cv_train_features, train_sentiments)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=5000,
                           n_iter_no_change=None, presort='auto',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [75]:
# predict on splitted test data
gbc_predictions = gbc_fit.predict(cv_test_features)

In [76]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, gbc_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, gbc_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

    Negative       0.96      0.94      0.95       297
    Positive       0.61      0.71      0.66        38

    accuracy                           0.92       335
   macro avg       0.79      0.83      0.81       335
weighted avg       0.92      0.92      0.92       335



,negative,positive
negative,280,17
positive,11,27


# XGBoost

In [77]:
import xgboost as xgb

xgb_fit = xgb.XGBClassifier(max_depth=2, n_estimators=5000, learning_rate=0.05)
xgb_fit.fit(cv_train_features, train_sentiments)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.05, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=5000, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [78]:
#conda install -c anaconda py-xgboost

In [79]:
#pip install xgboost

In [82]:
xgb_predictions = xgb_fit.predict(cv_test_features)

In [84]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, xgb_predictions))
pd.DataFrame(confusion_matrix(test_sentiments, xgb_predictions), index=labels, columns=labels)

              precision    recall  f1-score   support

    Negative       0.96      0.91      0.93       297
    Positive       0.50      0.68      0.58        38

    accuracy                           0.89       335
   macro avg       0.73      0.80      0.76       335
weighted avg       0.91      0.89      0.89       335



,negative,positive
negative,271,26
positive,12,26


In [85]:
xgb_test_tfpred = xgb_fit.predict(tv_test_data)

In [86]:
xgb_tfpred = pd.DataFrame(xgb_test_tfpred,columns= ["Sentiment"])
tfpred_result = pd.concat([TestData[["Review"]],xgb_tfpred] ,axis = 1)
#pred_result.to_csv('PredictedResult.csv',index = False)

In [87]:
tfpred_result['Sentiment'].value_counts()

Negative    41
Positive     9
Name: Sentiment, dtype: int64

In [89]:
gbc_test_tfpred = gbc_fit.predict(tv_test_data)

In [90]:
gbc_tfpred = pd.DataFrame(gbc_test_tfpred,columns= ["Sentiment"])
gbc_tfpred_result = pd.concat([TestData[["Review"]],gbc_tfpred] ,axis = 1)
#pred_result.to_csv('PredictedResult.csv',index = False)

In [91]:
gbc_tfpred_result['Sentiment'].value_counts()

Positive    50
Name: Sentiment, dtype: int64